In [2]:
import os
import pandas as pd
import time
import math
import re
import psycopg2 as pg

In [3]:
def findSynonyms (string, synonyms):
    synonymIndices = []
    synonymIDs = list(synonyms.id)
    synonymsName = list(synonyms.name)
    if string in synonymsName:
        index = synonymsName.index(string)
        idNumber = synonymIDs[index]
        for ind, row in synonyms.iterrows():
            if row['id']==idNumber:
                synonymIndices.append(ind)
        for i in synonyms.name[synonymIndices]:
            i = i.upper()
    return list(synonyms.name[synonymIndices])


def build_map(list1, list2):
    print("map builder not written yet")

def mergeDrugLists(list1,list2, synonyms):
    newList = []
    
    for item in list1:
        newList.append(item)
        
    for item in list2:
        if item in newList:
            print("found ", item, " in FDA resources")
        if item not in newList:
            syns = findSynonyms(item, synonyms)
            foundSynonym = False
            for syn in syns:
                if syn in newList:
                    found=True
                    print("found synonym for ", item, " in FDA resources")

            if not foundSynonym:
                print("could not find synonym for ", item, " in list 1. Adding it to the list.")
                newList.append(item)
    return newList

      
fda_ema_list = pd.read_excel("../merge_fda_ema/fda_ema_merged_list.xlsx")['Drug Name']
pmda_list = pd.read_excel("../pmda_ingest/pmda_list.xlsx")['Drug Name']
connection = pg.connect(database="drugcentral", user="drugman", password="dosage", host="unmtid-dbs.net", port=5433)
synonyms = pd.read_sql("""select synonyms.* from synonyms""", con=connection)
DrugCentralEntries = pd.read_sql("""select structures.* from structures""", con=connection)
synonyms['name'] = synonyms['name'].str.upper()
print(synonyms)

fda_ema_pmda_list = mergeDrugLists(fda_ema_list, pmda_list, synonyms)
sheetData = pd.DataFrame(data=[fda_ema_pmda_list]).transpose()
sheetData.columns = ['Drug Name']
sheetData.to_excel("fda_ema_pmda_merged_list.xlsx")

/var/folders/gm/d722cfgx1tdd8x4b5b1kd6j80000gn/T/ipykernel_53247/1157211217.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  synonyms = pd.read_sql("""select synonyms.* from synonyms""", con=connection)
/var/folders/gm/d722cfgx1tdd8x4b5b1kd6j80000gn/T/ipykernel_53247/1157211217.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DrugCentralEntries = pd.read_sql("""select structures.* from structures""", con=connection)


       syn_id      id                                  name  preferred_name  \
0       23310  5391.0                           SACITUZUMAB             NaN   
1       23311  5391.0                 SACITUZUMAB GOVITECAN             1.0   
2       23312  5391.0            SACITUZUMAB GOVITECAN-HZIY             NaN   
3       23313  5391.0                              TRODELVY             NaN   
4       23314  5391.0                              IMMU-132             NaN   
...       ...     ...                                   ...             ...   
23364   25485  5234.0  DURVALUMAB (GENETICAL RECOMBINATION)             NaN   
23365   25486  4210.0                                ELPLAT             NaN   
23366   25487  4384.0          CALCIUM LEVOFOLINATE HYDRATE             NaN   
23367   25488  4956.0              BEVACIZUMAB BIOSIMILAR 4             NaN   
23368   25489  5268.0  EMICIZUMAB (GENETICAL RECOMBINATION)             NaN   

       parent_id                                 ln